<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Inicialização de Bibliotecas, Pastas e Foco da Camera</p>

In [11]:
import cv2
import numpy as np
import os
import time
import functions as f
from collections import deque

os.makedirs("../data", exist_ok=True)
os.makedirs("../data/inputs", exist_ok=True)

for i in range(6):
    os.makedirs(f"../data/inputs/{i}", exist_ok=True)

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25)
cap.set(cv2.CAP_PROP_EXPOSURE, -1)

cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
cap.set(cv2.CAP_PROP_FOCUS, 10)

print("Exposição:", cap.get(cv2.CAP_PROP_EXPOSURE))
print("Foco:", cap.get(cv2.CAP_PROP_FOCUS))

Exposição: -6.0
Foco: -1.0


<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Bloco Principal</p>

In [ ]:
cap = cv2.VideoCapture('../data/.bin/sample.mp4')
bbox = False
grid = True
division = 16

while True:

    ret,frame_original = cap.read()

    if not ret:
        break

    # frame_original = f.mirroring(frame_original)
    frame = frame_original.copy()
    frame = f.gray_scale(frame)
    frame = f.background_subtraction(cam_frame=frame,start_time=3,limiar=15)
    sample,frame = f.down_sampling(frame,division=division)
    frame = f.binarization(frame,limiar=5)
    sample = f.binarization(sample,limiar=5)
    bbox_points,frame_original = f.bounding_box(frame,frame_vizualizer=frame_original,division=division)

    vector = f.vectorize(sample, bbox_points=bbox_points, division=division)

    print(np.sum(vector))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    elif cv2.waitKey(1) & 0xFF == ord('1'):
        f.background_subtraction.background = f.gray_scale(frame_original.copy())
        frame_original = np.full_like(frame_original,255)
        print('Background reset')


    cv2.imshow('Down Sampled', sample)
    cv2.imshow('Visualizer',frame)
    cv2.imshow('Original',frame_original)
    


cap.release()
cv2.destroyAllWindows()

1073
1069
1066
1065
1064
1064
1068
1068
1068
1066
1066
1065
1065
1068
1068
1062
1062
1068
1068
1062
1062
1066
1066
1067
1067
1067
1061
1061
1067
1065
1065
1065
1065
1065
1068
1068
1062
1062
1068
1068
1064
1064
1066
1066
1066
1066
1066
1067
1065
1065
1065
1065
1065
1065
1064
1064
1067
1067
1062
1062
1062
1069
1067
1067
1067
1067
1067
1078
1078
1072
1072
1072
1072
1078
1078
1068
1068
1085
1085
1073
1074
1078
1069
1069
1069
1080
1080
1067
1067
1074
1074
1078
1078
1081
1068
1068
1077
1077
1077
1077
1077
1075
1081
1081
1081
1070
1070
1074
1072
1072
1078
1078
1069
1069
1082
1082
1082
1071
1071
1073
1073
1077
1077
1070
1070
1079
1079
1070
1070
1076
1075
1073
1081
1081
1070
1070
1070
1078
1078
1075
1075
1069
1069
1081
1081
1069
1069
1079
1079
1073
1075
1080
1080
1069
1069
1080
1080
1070
1070
1076
1076
1077
1077
1072
1072
1080
1080
1070
1070
1076
1076
1076
1070
1081
1081
1067
1067
1078
1078
1075
Background reset
50
102
102
119
119
119
128
127
127
110
110
110
109
109
121
121
118
118
105
105
110


<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Bloco Video REC
</p>

In [ ]:
gravador = cv2.VideoWriter('../data/.bin/asd.avi',fourcc=cv2.VideoWriter_fourcc(*'XVID'),fps=30qqqqqqqq,frameSize=(640,480))
cam = cv2.VideoCapture(0)
rec = False
while True:
    ret,original_frame = cam.read()
    original_frame = f.mirroring(original_frame)

    if not ret:
        break
    
    frame = original_frame.copy()
    frame = f.gray_scale(frame)
    frame = f.background_subtraction(cam_frame=frame,start_time=3,limiar=15)
    vector,frame = f.down_sampling(frame,division=16)
    frame = f.binarization(frame,limiar=5)
    vector = f.binarization(vector,limiar=5)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    elif cv2.waitKey(1) & 0xFF == ord('1'):
        f.background_subtraction.background = f.gray_scale(original_frame.copy())
        frame_original = np.full_like(original_frame,255)
    elif cv2.waitKey(1) & 0xFF == ord('2'):
        rec = True
        print('gravandooo...')

    if rec:
        frame = np.stack([frame]*3, axis=-1)
        gravador.write(frame)


    cv2.imshow('Original',original_frame)
    cv2.imshow('Visualizer',frame)

cam.release()
gravador.release()
cv2.destroyAllWindows()